In [2]:
import os
from mayavi import mlab
from tvtk.tvtk_access import tvtk
import hermes
from hermes.analysis import LOSAnalysis
from hermes.objects import Satellite, SatGroup, Earth, Constellation
from hermes.scenario import Scenario
from hermes.simulation import Simulation
from hermes.util import hex2rgb

from astropy import units as u, time
import numpy as np

# PDF exporter defaults
ex = tvtk.GL2PSExporter()
ex.file_format = 'pdf'
ex.sort = 'bsp'
ex.compress = 1
#ex.edit_traits(kind='livemodal')

%matplotlib inline
%load_ext autoreload

%autoreload 2

gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


### Example constellation

In [3]:
def _Example():

    n_pol_planes = 6
    pol_set = SatGroup.as_set(Earth.poli_body,
                            a=Earth.poli_body.R_mean + 1000 * u.km, ecc=0 * u.one, inc=99.5 * u.deg,
                            rraan=np.array([0, 63.2, 94.8, 126.4, 158.0, 31.6]) * u.deg,
                            aargp=np.repeat(0 * u.deg, n_pol_planes),
                            nnnu=[np.arange(345, 0, -30) * u.deg] * n_pol_planes)
    pol_set.color = hex2rgb("#0074D9")
    pol_set.fov = 45 * u.deg

    n_inc_planes = 5
    inc_set = SatGroup.as_set(Earth.poli_body,
                            a=Earth.poli_body.R_mean + 1248 * u.km, ecc=0 * u.one, inc=37.4 * u.deg,
                            rraan=np.array([0, 36.0, 72.0, 108.0, 144.0]) * u.deg,
                            aargp=np.repeat(0 * u.deg, n_inc_planes),
                            nnnu=[np.arange(320, -10, -40) * u.deg] * n_inc_planes)
    inc_set.color = hex2rgb("#FF4136")
    inc_set.fov = 45 * u.deg

    constellation = Constellation(name='Example')
    constellation.append(pol_set)
    constellation.append(inc_set)

    return constellation

Example = _Example()

C:\git\hermes-optimization\cenv\lib\site-packages\astropy\units\decorators.py:222: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)


In [4]:
start = time.Time('2019-09-01 10:00:00.000', scale='tt')        # Start time of simulation
stop = time.Time('2019-09-02 10:00:00.000', scale='tt')         # Stop time of simulation
step = 1 * u.s

scenario = Scenario(start, stop, step, hermes.objects.Earth, name='Example')

# Make a new Satellite object and give it a cyan color
sat_400_51 = Satellite.circular(hermes.objects.Earth.poli_body, 400 * u.km, inc=51.6 * u.deg, raan=0 * u.deg, arglat=0 * u.deg)
sat_400_51.color = hex2rgb('#00ffff')
sat_400_51.name = 'sat_400km_51deg'
#sat_400_51.J2_perturbation = True

sat_500_97 = Satellite.circular(hermes.objects.Earth.poli_body, 500 * u.km, inc=90.0 * u.deg, raan=0 * u.deg, arglat=0 * u.deg)
sat_500_97.color = hex2rgb('#00ffff')
sat_500_97.name = 'sat_500km_97deg'
#sat_500_97.J2_perturbation = True

# And we add it to the scenario
#scenario.add_satellite(sat_400_51)
scenario.add_satellite(sat_500_97)

# Add the constellation
constellation = Example
scenario.add_satellite(constellation)

# Add line-of-sight analysis
#scenario.add_analysis(LOSAnalysis(scenario, sat_400_51, constellation))
scenario.add_analysis(LOSAnalysis(scenario, sat_500_97, constellation))

# Initialise the scenario
scenario.initialise()

simulation = Simulation(scenario, show_3d=False)
simulation.run()

Initializing attractor
Initializing 118 satellites...
Initializing analysis 1 of 1...


Simulation progress: 100%|██████████| 86400/86400 [00:55<00:00, 1569.80it/s]


In [2]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z']

file_path = "2021-02-22_09-53-18_Example_LOSAnalysis_sat_500km_97deg_to_Example.h5"

row_limit = -1

import pandas as pd

store_500km = pd.HDFStore(file_path)
instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)

from hermes.postprocessing import *

# Add range column d
instances_500km_df = add_range(instances_500km_df)

# Add sff
instances_500km_df = add_sff(instances_500km_df)

instances_500km_df.to_hdf('example_data.h5', 'contact_instances', format='table')